<a href="https://colab.research.google.com/github/PedroChumpitaz/A-Multilingual-Human-Agent-Interaction-System-for-Enhanced-International-Dining-Experiences/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lifelines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 7.8 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=38f3367cf1cc2d39710f79becdf1a24c2b782dac69adc5dfb211844a93be92e5
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from lifelines import KaplanMeierFitter

# Funciones para cargar datos de cada módulo
def mean_survival_time_km(group):
    T = group['TVIDA']
    E = np.ones_like(T)  # Suponemos que todos los eventos están censurados

    kmf = KaplanMeierFitter()
    kmf.fit(T, event_observed=E, label='Kaplan-Meier')

    kmf_times = kmf.survival_function_.index
    kmf_probabilities = kmf.survival_function_['Kaplan-Meier']
    mean_survival_time = np.sum(np.diff(kmf_times) * kmf_probabilities.iloc[:-1])

    return mean_survival_time

def cargar_datos_mod_18(year):
    ruta = f'/content/drive/MyDrive/Survival-modeling-2024/data/mod-18/Enaho01-{year}.csv'
    df = pd.read_csv(ruta, encoding='latin1')
    df = df[df['P612N'] == 2]
    variables_cualitativas = ['AÑO', 'MES', 'CONGLOME', 'VIVIENDA', 'HOGAR', 'UBIGEO', 'DOMINIO', 'ESTRATO', 'P612N', 'P612', 'P612B', 'P612C', 'P612C1', 'FACTOR07', 'NCONGLOME']
    variables_cuantitativas = ['P612A', 'P612G', 'P612H', 'D612G', 'D612H', 'I612G', 'I612H', 'TICUEST01']
    variables_cualitativas = [col for col in variables_cualitativas if col in df.columns]
    variables_cuantitativas = [col for col in variables_cuantitativas if col in df.columns]
    df = df[variables_cualitativas + variables_cuantitativas]
    for col in variables_cuantitativas:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df['AÑO'] = pd.to_numeric(df['AÑO'], errors='coerce')
    df['P612C'] = pd.to_numeric(df['P612C'], errors='coerce')
    valores_a_eliminar = [99, 999, 9999, 99999, 999999, 999999.99, 9]
    for valor in valores_a_eliminar:
        df = df.replace(valor, np.nan)
    df = df.dropna(subset=['P612C'])

    df['TVIDA'] = df['AÑO'] - df['P612C']
    df = df[df['TVIDA'] > 0]

    grouped = df.groupby('UBIGEO')

    resumen_cualitativo = grouped[variables_cualitativas].agg(moda)
    resumen_cuantitativo = grouped[variables_cuantitativas].agg(mediana)

    # Calcular el tiempo medio de supervivencia para cada grupo y agregarlo al resumen
    mean_survival_times = grouped.apply(mean_survival_time_km)
    mean_survival_times.name = 'mean_survival_time_km'

    resumen = pd.concat([resumen_cualitativo, resumen_cuantitativo, mean_survival_times], axis=1)
    columnas_seleccionadas = ['AÑO', 'P612A', 'P612B', 'P612C', 'FACTOR07', 'P612G', 'P612H', 'D612G', 'D612H', 'I612G', 'I612H', 'mean_survival_time_km']
    resumen = resumen[columnas_seleccionadas]
    resumen = resumen.reset_index()
    resumen = resumen.rename(columns={'FACTOR07': 'FACTOR07_MOD18'})
    return resumen

def cargar_datos_mod_32(year):
    ruta_sumaria = f'/content/drive/MyDrive/Survival-modeling-2024/data/mod-32/Sumaria-{year}.csv'
    df = pd.read_csv(ruta_sumaria, encoding='latin1')
    col_m32 = ['AÑO', 'UBIGEO', 'INGMO2HD', 'FACTOR07', 'SG42', 'SG42D']
    df = df[col_m32]
    variables_cualitativas = ['AÑO']
    variables_cuantitativas = ['INGMO2HD', 'FACTOR07']
    variables_excluyentes = ['SG42', 'SG42D']
    for col in variables_cuantitativas + variables_excluyentes:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df['AÑO'] = pd.to_numeric(df['AÑO'], errors='coerce')
    grouped = df.groupby('UBIGEO')
    resumen_cualitativo = grouped[variables_cualitativas].agg(moda)
    resumen_cuantitativo = grouped[variables_cuantitativas].agg(mediana)
    resumen_excluyente = grouped[variables_excluyentes].agg(mediana_excluyendo_ceros)
    resumen = pd.concat([resumen_cualitativo, resumen_cuantitativo, resumen_excluyente], axis=1)
    resumen = resumen.reset_index()
    resumen = resumen.rename(columns={'FACTOR07': 'FACTOR07_SUMARIA'})
    return resumen

def cargar_datos_mod_01(year):
    ruta_sumaria = f'/content/drive/MyDrive/Survival-modeling-2024/data/mod-1/Enaho01-{year}.csv'
    df = pd.read_csv(ruta_sumaria, encoding='latin1')
    variables_cualitativas = ['AÑO', 'P101', 'P102', 'P103', 'P103A', 'P1143', 'P1144']
    variables_cuantitativas = ['P104']
    for col in variables_cuantitativas:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df['AÑO'] = pd.to_numeric(df['AÑO'], errors='coerce')
    grouped = df.groupby('UBIGEO')
    resumen_cualitativo = grouped[variables_cualitativas].agg(moda)
    resumen_cuantitativo = grouped[variables_cuantitativas].agg(mediana)
    resumen = pd.concat([resumen_cualitativo, resumen_cuantitativo], axis=1)
    resumen = resumen.reset_index()
    return resumen

def cargar_datos_mod_05(year):
    ruta_sumaria = f'/content/drive/MyDrive/Survival-modeling-2024/data/mod-5/Enaho01-{year}.csv'
    df = pd.read_csv(ruta_sumaria, encoding='latin1')
    variables_cualitativas = ['AÑO', 'P510']
    df['AÑO'] = pd.to_numeric(df['AÑO'], errors='coerce')
    df['P510'] = pd.to_numeric(df['P510'], errors='coerce')
    grouped = df.groupby('UBIGEO')
    resumen_cualitativo = grouped[variables_cualitativas].agg(moda_excluyendo_ceros)
    resumen_cualitativo = resumen_cualitativo.reset_index()
    return resumen_cualitativo

def cargar_datos_mod_03(year):
    ruta_sumaria = f'/content/drive/MyDrive/Survival-modeling-2024/data/mod-3/Enaho01-{year}.csv'
    df = pd.read_csv(ruta_sumaria, encoding='latin1')
    variables_cualitativas = ['AÑO', 'P301A']
    df['AÑO'] = pd.to_numeric(df['AÑO'], errors='coerce')
    df['P301A'] = pd.to_numeric(df['P301A'], errors='coerce')
    grouped = df.groupby('UBIGEO')
    resumen_cualitativo = grouped[variables_cualitativas].agg(moda_excluyendo_ceros)
    resumen_cualitativo = resumen_cualitativo.reset_index()
    return resumen_cualitativo

# Definir funciones de agregación
def moda(series):
    series = series.dropna()
    return series.mode().iloc[0] if not series.mode().empty else np.nan

def mediana(series):
    series = series.dropna()
    return series.median() if not series.empty else np.nan

def mediana_excluyendo_ceros(series):
    series = series.replace(0, np.nan).dropna()
    return series.median() if not series.empty else np.nan

def moda_excluyendo_ceros(series):
    series = series.replace(0, np.nan).dropna()
    return series.mode().iloc[0] if not series.mode().empty else np.nan

# Procesar los datos de cada módulo para los años 2008 a 2023 y concatenar los resultados
all_years_data = []

for year in range(2016, 2023 + 1):
    resumen_mod18 = pd.DataFrame()
    resumen_mod32 = pd.DataFrame()
    resumen_mod01 = pd.DataFrame()
    resumen_mod05 = pd.DataFrame()
    resumen_mod03 = pd.DataFrame()

    resumen_mod18 = cargar_datos_mod_18(year)
    resumen_mod32 = cargar_datos_mod_32(year)
    resumen_mod01 = cargar_datos_mod_01(year)
    resumen_mod05 = cargar_datos_mod_05(year)
    resumen_mod03 = cargar_datos_mod_03(year)

    # Fusionar los DataFrames usando 'UBIGEO' y 'AÑO'
    df_combinado = resumen_mod18.merge(resumen_mod32, on=['UBIGEO', 'AÑO'], how='inner')
    df_combinado = df_combinado.merge(resumen_mod01, on=['UBIGEO', 'AÑO'], how='inner')
    df_combinado = df_combinado.merge(resumen_mod05, on=['UBIGEO', 'AÑO'], how='inner')
    df_combinado = df_combinado.merge(resumen_mod03, on=['UBIGEO', 'AÑO'], how='inner')

    all_years_data.append(df_combinado)

# Concatenar todos los resultados en un único DataFrame
df_final = pd.concat(all_years_data, ignore_index=True)

# Cargar DataFrame de ubigeo desde el archivo
ruta_ubigeo = '/content/drive/MyDrive/Survival-modeling-2024/data/ubigeo_lima.csv'
ubigeo = pd.read_csv(ruta_ubigeo, encoding='latin1')

# Filtrar los datos para Lima usando el DataFrame de ubigeo
df_lima = df_final[df_final['UBIGEO'].isin(ubigeo['UBIGEO'])]

# Agregar columnas de "DISTRITO", "PROVINCIA", "PERFIL ZONAL" del DataFrame ubigeo
df_lima = df_lima.merge(ubigeo[['UBIGEO', 'DISTRITO', 'PROVINCIA', 'PERFIL ZONAL']], on='UBIGEO', how='left')

<ipython-input-5-86c6f9ff2f02>:93: DtypeWarning: Columns (212,217,222,230,235,240,245,250,255,262,265,268,271,274,277,280,283,287,289,291,293,295,297,299,302,303,304,305,306,310,311,397,398,399,400,517,518,519,520,521,522,523,524,525,526,597,600,605,768,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_sumaria, encoding='latin1')
<ipython-input-5-86c6f9ff2f02>:104: DtypeWarning: Columns (13,14,27,33,44,213,228,295) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_sumaria, encoding='latin1')
<ipython-input-5-86c6f9ff2f02>:21: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, encoding='latin1')
<ipython-input-5-86c6f9ff2f02>:93: DtypeWarning: Columns (20,21,22,23,24,25,26,27,28,29,30,31,32,33,201,202,203,208,212,217,222,230,235,240,245,250,255,262,26

In [ ]:
# Especifica la ruta donde quieres guardar el archivo
file_path = '/content/drive/MyDrive/Survival-modeling-2024/df_lima.csv'

# Guarda el DataFrame como un archivo CSV
df_lima.to_csv(file_path, index=False)